In [8]:
import olefile
import zlib
import struct
import re

def get_hwp_text(filename):
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    if ["FileHeader"] not in dirs or ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):])) 

    sections = ["BodyText/Section" + str(x) for x in sorted(nums)]

    text = ""
    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed:
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data

        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            # Handle text records (rec_type == 67)
            if rec_type in [67]:
                rec_data = unpacked_data[i + 4:i + 4 + rec_len]
                try:
                    section_text += rec_data.decode('utf-16le')
                except UnicodeDecodeError:
                    section_text += "[DECODE ERROR]"
            # Handle paragraph (rec_type == 66), which may represent new lines or paragraph breaks
            elif rec_type == 66:
                section_text += "\n"  # Add a line break for paragraph break

            i += 4 + rec_len

        text += section_text
        text += "\n"

    # 공백 및 제어 문자 제거
    text = text.strip()

    # 한글 및 ASCII 문자만 유지, 그 외의 문자는 제거 (중국어 및 특수문자 제거)
    # 줄바꿈(\n)은 유지
    text = re.sub(r'[^\uAC00-\uD7A3\u0000-\u007F\n\s]', '', text)

    # 추가로 제어 문자 (ASCII 코드 0-31 사이의 문자) 제거
    text = re.sub(r'[\x00-\x1F]+', '', text)

    # 학칙 항목별로 줄바꿈을 추가하는 로직
    # 일반적으로 학칙은 "제XX조"와 같은 패턴이 포함될 수 있음
    text = re.sub(r'(제\d+조)', r'\n\1', text)  # "제XX조" 패턴 앞에 줄바꿈 추가
    text = re.sub(r'(제\d+장)', r'\n\1\n', text)  # "제XX장" 패턴 앞뒤에 줄바꿈 추가

    # 날짜 패턴 줄바꿈 추가 (YYYY. MM. DD 형식)
    date_pattern = r'(\d{4}\. \d{2}\. \d{2})'
    text = re.sub(date_pattern, r'\n\1', text)  # 날짜 패턴 앞에 줄바꿈 추가

    # 날짜에 맞는 위치로 이동시키기 위한 추가 로직
    # 각 날짜와 관련된 텍스트를 적절히 분리하여 이동
    # 개정 관련 부분은 보통 "개정"과 날짜가 붙어 있기 때문에 이를 고려한 로직 추가
    text = re.sub(r'(개정\s*\d{4}\. \d{2}\. \d{2})', r'\n\1', text)

    return text

# Replace 'sample2.hwp' with your HWP file path
text = get_hwp_text('sample2.hwp')

# Output the formatted text
print(text)

# Write to a file
with open('Example.txt', 'w', encoding='utf-8') as file:
    file.write(text)


한국폴리텍대학 학칙 제정 
1998. 02. 11
개정 
2000. 03. 01
개정 
2000. 09. 14
개정 
2001. 03. 01
개정 
2002. 03. 01
개정 
2003. 03. 01
개정 
2004. 03. 01
개정 
2005. 03. 01
개정 
2005. 04. 21
개정 
2006. 03. 01
개정 
2007. 03. 01
개정 
2007. 08. 20
개정 
2008. 03. 01
개정 
2009. 03. 01
개정 
2010. 01. 01
개정 
2010. 03. 01
개정 
2010. 09. 01
개정 
2011. 03. 01
개정 
2012. 03. 01
개정 
2012. 07. 03
개정 
2013. 03. 01
개정 
2014. 03. 01
개정 
2015. 03. 01
개정 
2016. 03. 01
개정 
2017. 03. 01
개정 
2018. 02. 06
개정 
2018. 02. 22
개정 
2019. 02. 08
개정 
2019. 03. 06
개정 
2019. 04. 25
개정 
2019. 08. 01
개정 
2019. 09. 23
개정 
2020. 03. 01
개정 
2020. 08. 20
개정 
2021. 03. 01
제1장
 총    칙
제1조(목적) 이 학칙은 한국폴리텍대학(이하 대학이라 한다)의 교육목표를 설정하고 이를 달성하기 위한 학사운영 등에 관한 사항을 규정함을 목적으로 한다. <개정 12.3.1, `18.2.6>
제2조(교육목표) 대학은 산업현장에서 필요로 하는 기술기능 인력을 양성하고 근로자 평생직업능력개발, 산학협력 및 산업인력개발 등을 통하여 국가발전 및 지역산업체의 인적기술적 지원에 기여함을 교육목표로 한다. <신설 12.3.1>
제3조(설치과정)  본 대학은 2년제학위과정(다기능기술자), 학위전공심화과정, 직업훈련과정을 설치하고자 할 때에는 사회적 수요, 국가 장기발전계획, 교육훈련과정 성격 및 본 대학의 교육목표 등에 비추어 그 필요성이 인정되는 과정을 설치할 수 있다. <신설 12.3